<a href="https://colab.research.google.com/github/laurasels/workshop_satellietbeelden/blob/master/Workshop_Satellietdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Thema sessie NWB: Workshop Satellietdata**
## Het detecteren van mutaties in het NWB met behulp van satellietdata en beeldherkenning.

Deze workshop zal bestaan uit 3 delen:
- Het verzamelen van de satellietbeelden en deze opknippen
- koppeling naar het NWB
- beeldherkenning voor rotondes middels een Machine Learning algoritme

## **Voorbereiding**

Voordat we beginnen, moeten we de data uploaden. Het zip bestand dat je hebt ontvangen via wetransfer kun je hier uploaden.

Het zip bestand bevat onder andere een satellietafbeelding, een NWB bestand en een beeldherkenningsmodel.

In [0]:
from google.colab import files
files.upload()

In [0]:
!curl https://transfer.sh/j4Ena/workshop.zip -o workshop.zip

Dan unzippen we het bestand door *!unzip bestandsnaam.zip*

In [0]:
!unzip workshop.zip #bestandsnaam hier aanpassen

## **Stap 1: satellietdata**

De satellietdata wordt verkregen via https://www.satellietdataportaal.nl/.




Omdat deze bestanden erg groot zijn om te downloaden, hebben we alvast een plaatje van Maastricht gedownload.


In [0]:
import matplotlib.pyplot as plt
from matplotlib import image

plt.figure(figsize=(12,12)) #grootte van het plotje
img = image.imread("workshop/Maastricht.tif")
plt.imshow(img)

plt.grid()

In [0]:
print("Pixels afbeelding Maastricht: ", img.shape[0],"x", img.shape[1])

Nog steeds is de afbeelding te groot om beeldherkenning toe te passen. Daarom knippen we de afbeelding op in kleinere afbeeldingen van 1000x1000 pixels. 

Om dit te doen gebruiken we bepaalde python packages. Het volgende gedeelte runnen we zodat deze packages worden geladen. Wij maken gebruik van GDAL (Geospatial Data Abstraction Library). GDAL werkt goed met raster (.tif) en  vector (.shp) bestanden.

In [0]:
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y
print("All done.")

Nu kunnen we gaan knippen!

In [0]:
import os, gdal

in_path = "workshop/Maastricht.tif"

os.makedirs("workshop/satelliet", exist_ok=True) # hier maken we een nieuwe map 'satelliet' waar de plaatjes straks in komen
out_path = "workshop/satelliet/tile_"

tile_size_x = 1000 #aantal pixels breedte
tile_size_y = 1000 #aantal pixels lengte

ds = gdal.Open(in_path)
band = ds.GetRasterBand(1)
xsize = band.XSize
ysize = band.YSize

for i in range(0, xsize, tile_size_x):
    for j in range(0, ysize, tile_size_y):
        com_string = "gdal_translate -of GTIFF -srcwin " + str(i)+ ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(in_path) + " " + str(out_path) + str(i) + "_" + str(j) + ".tif"
        os.system(com_string)

print("All done.")

Laten we een paar voorbeelden bekijken.

In [0]:
plt.rcParams["figure.figsize"] = (8,8)

img = image.imread("workshop/satelliet/tile_3000_1000.tif")
plt.imshow(img)
plt.show()

img = image.imread("workshop/satelliet/tile_2000_2000.tif")
plt.imshow(img)
plt.show()

img = image.imread("workshop/satelliet/tile_5000_1000.tif")
plt.imshow(img)
plt.show()

## **Stap 2: NWB**

Nu we de satellietbeelden hebben opgeknipt, willen we aan elk satelliet tegeltje het NWB bestand linken. 

Dit doen we door voor elke geknipte tegel, de tegel 'leeg' te openen en hierover het NWB bestand te openen. De wegen printen we in lichtgrijs (kleurcode 200) en de rotondes in zwart (kleurcode 1), zodat er geen verwarring kan ontstaan over wat een rotonde is. Het resultaat slaan we vervolgens op als .tif bestand.

We definiëren de volgende functie die dit proces voor ons zal doen.

In [0]:
# A script to rasterise a shapefile to the same projection & pixel resolution as a reference image.
from osgeo import ogr, gdal
import os
import subprocess
import numpy as np

#shp_naar_tif.py maar dan als een functie op te roepen door rasterize(-,-,-,-)
def rasterize(InputVector, InputVectorRB, RefImage, OutputImage):   
    gdalformat = 'GTiff'
    datatype = gdal.GDT_Byte
    burnVal = 1 #value for the output image pixels
	##########################################################
	# Get projection info from reference image
    Image = gdal.Open(RefImage, gdal.GA_ReadOnly)
  # Get RGB array of reference image
    band = Image.GetRasterBand(1)
    xsize = band.XSize
    ysize = band.YSize
    rgb = np.dstack([Image.GetRasterBand(b).ReadAsArray() for b in (1,2,3)])
    x,y,z = rgb.shape    
	# Open Shapefile
    Shapefile = ogr.Open(InputVector)
    Shapefile_layer = Shapefile.GetLayer()
  # Open RB Shapefile
    ShapefileRB = ogr.Open(InputVectorRB)
    ShapefileRB_layer = ShapefileRB.GetLayer()
	# Rasterise
    print("Rasterising shapefile...")
    Output = gdal.GetDriverByName(gdalformat).Create(OutputImage, Image.RasterXSize, Image.RasterYSize, 1, datatype, options=['COMPRESS=DEFLATE'])
    Output.SetProjection(Image.GetProjectionRef())
    Output.SetGeoTransform(Image.GetGeoTransform()) 
	# Write data to band 1
    Band = Output.GetRasterBand(1)
    Band.SetNoDataValue(255)
    gdal.RasterizeLayer(Output, [1], Shapefile_layer, burn_values=[200])
    gdal.RasterizeLayer(Output, [1], ShapefileRB_layer, burn_values=[burnVal])
	# Prints black part on the new output image if the reference image is partially black (border image)
    a = np.count_nonzero(rgb==0)
    if a >= (rgb.size*0.07):
        bw = Output.GetRasterBand(1).ReadAsArray()
    	#compare
        for i in range(x):
            for j in range(y):
                if max(rgb[i,j,:])==0:
                    bw[i,j]=0       
        Output.GetRasterBand(1).WriteArray(bw)	
	# Close datasets
    Band = None
    Output = None
    Image = None
    Shapefile = None
  # Build image overviews
    subprocess.call("gdaladdo --config COMPRESS_OVERVIEW DEFLATE "+OutputImage+" 2 4 8 16 32 64", shell=True)
    print("Done.")

print("Functie is gedefinieerd.")

Nu kunnen we de functie aanroepen om satelliet tegels om te zetten in NWB tegels. De functie heeft 4 argumenten die we eerst moeten definiëren.

In [0]:
InputVector = "workshop/NWB_Maastricht/Maastricht_NWB.shp" # pad naar het NWB van Maastricht
InputVectorRB = "workshop/NWB_Maastricht/Maastricht_RB.shp" # pad naar het NWB van Maastricht met alleen rotondes

RefImage = "workshop/satelliet/tile_3000_1000.tif" # pad naar de satelliettegel die omgezet moet worden

os.makedirs("workshop/NWB", exist_ok=True) #maak een nieuwe map waar de NWB tegels in komen
OutputImage = os.path.join("workshop/NWB/NWB_"+RefImage.rsplit('/')[-1]) #pad naar de nieuwe NWB tegel met dezelfde naam als de satelliettegel

print(RefImage, " -> ",OutputImage)
rasterize(InputVector, InputVectorRB, RefImage, OutputImage)

Laten we eens kijken hoe de NWB tegel er uit ziet.

In [0]:
plt.figure(figsize=(15,15))

plt.subplot(121)
img = image.imread(RefImage)
plt.imshow(img)

plt.subplot(122)
img = image.imread(OutputImage)
plt.imshow(img, cmap='gray', vmin=0, vmax=255)

plt.show()

We zien dat de rotonde in het zwart is geprint en de wegen in grijs. 

Dit willen we eigenlijk voor alle afbeeldingen in de map satelliet doen. Het volgende script 'wandelt' langs alle afbeeldingen in de map satelliet en roept hiervoor de eerder gemaakte functie *rasterize()* aan.

In [0]:
for subdir, dirs, files in os.walk("workshop/satelliet"):
  for name in files:
    RefImage = os.path.join(subdir+"/"+name)
    OutputImage = os.path.join("workshop/NWB/NWB_"+RefImage.rsplit('/')[-1])
    rasterize(InputVector,InputVectorRB,RefImage,OutputImage)

print("All done.")

Laten we nog wat voorbeelden plotten.

In [0]:
plt.rcParams["figure.figsize"] = (8,8)

img = image.imread("workshop/NWB/NWB_tile_3000_1000.tif")
plt.imshow(img, cmap="gray", vmin=0, vmax=255)
plt.show()

img = image.imread("workshop/NWB/NWB_tile_2000_2000.tif")
plt.imshow(img, cmap="gray", vmin=0, vmax=255)
plt.show()

img = image.imread("workshop/NWB/NWB_tile_5000_1000.tif")
plt.imshow(img, cmap="gray", vmin=0, vmax=255)
plt.show()

## **Stap 3: beeldherkenning**

De data staat klaar in de mappen satelliet en NWB. Op deze afbeeldingen kunnen we nu beeldherkenning toepassen. Dit doen we door het getrainde model te laden. In de summary() staan alle lagen die het model heeft. We zien in de eerste laag als input een afbeelding van 1000 x 1000 pixels. In de allerlaatste laag (Dense1) zien we dat de input is omgezet in één score: een score boven de 50% resulteert in de klasse 1 (rotonde) en een score beneden de 50% resulteert in de klasse 0 (geen rotonde).


In [0]:
from keras.models import load_model
model = load_model('workshop/finalmodel2.h5', compile=False)

model.summary()

Het model is al getraind, dus we kunnen direct beginnen met voorspellen! Hiervoor laden we weer wat python packages in en definiëren we een functie die de afbeelding plot en de score eronder weergeeft.

In [0]:
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.vgg16 import preprocess_input

def predict_image(file):
  x = load_img(file, target_size=(1000,1000))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  score = model.predict(x)[0][0]
  img = image.imread(file)
  plt.rcParams["figure.figsize"] = (7,7)
  plt.figure()
  plt.imshow(img)
  plt.xlabel("Predicted score: "+ str(score))
  plt.show()

We kunnen nu de functie *predict_image()* aanroepen met als argument het pad naar afbeelding.

In [0]:
predict_image("workshop/satelliet/tile_3000_1000.tif")
predict_image("workshop/satelliet/tile_2000_2000.tif")
predict_image("workshop/satelliet/tile_5000_1000.tif")

Voor het NWB bestand hebben we ook nog een voorspelling nodig. Aangezien we al weten waar de rotondes zijn, hoeven we hier niet per se beeldherkenning op toe te passen (maar dat mag natuurlijk wel). 

De volgende functie maakt gebruik van de kleurcode van de geprinte rotondes. Wanneer een afbeelding de kleurcode 1 bevat, wordt de klasse 1 (rotonde) weergegeven. Wanneer de afbeelding geen zwart geprinte rotonde bevat en dus geen kleurcode 1, wordt de klasse 0 (geen rotonde) weergegeven.

In [0]:
def predict_NWB(file):
  Image = gdal.Open(file)
  rgb = np.dstack([Image.GetRasterBand(1).ReadAsArray()])
  a = np.count_nonzero(rgb==1) #telt alle pixels met kleurcode 1 in de afbeelding
  if a>0:
    class_NWB=1
  else:
    class_NWB=0
  img = image.imread(file)
  plt.rcParams["figure.figsize"] = (7,7)
  plt.figure()
  plt.imshow(img,cmap="gray", vmin=0, vmax=255)
  plt.xlabel("Predicted score: "+ str(class_NWB))
  plt.show()

In [0]:
predict_NWB("workshop/NWB/NWB_tile_3000_1000.tif")
predict_NWB("workshop/NWB/NWB_tile_2000_2000.tif")
predict_NWB("workshop/NWB/NWB_tile_5000_1000.tif")

Vervolgens kunnen we de plaatjes onder elkaar plotten voor meer overzicht.

In [0]:
tile = "tile_2000_2000.tif"
predict_image("workshop/satelliet/"+tile)
predict_NWB("workshop/NWB/NWB_"+tile)


## **Conclusie**

In Maastricht worden alle rotondes (9 stuks op dit deel van Maastricht) herkent met het algoritme. In het NWB bestand zijn al deze rotondes al geregistreerd.

Maastricht heeft dus weinig aan het model, maar in Zuid-Holland zijn tot nu **aantal** rotondes gevonden die nog niet bekend waren bij het NWB. 

Het model geeft ook een aantal 'false positives' weer. Dit zijn delen die worden herkent als rotonde, maar het in werkelijkheid niet zijn. Vaak zijn dit ronde vormen in wegen of gebouwen. In de toekomst kan het model met meer data nog beter getraind worden zodat deze false positives minder vaak voor komen. Daarnaast kan het model nog uitgebreid worden naar het herkennen van nieuwe wegen of rijrichtingen. Suggesties zijn uiteraard ook welkom.